# Deep Convolutional Generative Adversarial Network  (1of2)

**OVERVIEW**

This model uses the parameters as in the 'CHANGE' box + *ReLU* as activation layer in both the gen. and disc. 

# CHANGE

_____________________________________________________________________

In [ ]:
# Image size (height x width)
ih = 64
iw = 64

# Grayscale or RGB
ch = 'grayscale'

# Layer adapt
ksize = 4 # Kernel size : was '4' for 64x64 image
ssize = 2 # Stride size : was '2' for 64x64 image

# Batch size
batch_size = 32

# Number of epochs
epoch_t = 5

# Where computation is performed: Kaggle (0) or Local (1)
cenv = 0

# Do you want to restore from checkpoint
restore = False

# Are you going to use a TPU?
use_tpu = 'n'

_____________________________________________________________________

In [ ]:
if cenv == 0:
    print("Computation environment: Kaggle")
if cenv == 1:
    print("Computation environment: Local")

## Setup

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import cv2
import random as rd
import PIL

**Create directories for saving output**

In [ ]:
if cenv == 1:
    file_exists = []
    vnum = 1
    dir = "C:/Users/Max/Documents/GitHub/dcgan"
    for files in os.listdir(dir):
        if "dcgan" in files: 
            try:
                vnum = max(vnum, int(files[-3:]))
            except: 
                continue
            new_vnum = vnum + 1
            file_exists.append(True)
        else: 
            file_exists.append(False)
    # If this is the first notebook you want to save, a new folder will be created with version #001
    if sum(file_exists) == 0:
        new_vnum = 1
        print("No matches found")

    else: 
        print(f"{sum(file_exists)} matches(es) found")
        print("--------------")

    # Print new folder name
    print(f"New folder name: dcgan-local-v{new_vnum:03}")
    print("--------------")
    
    # Create new folder with the name of the notebook and the version number
    new_dir = f"/Users/Max/Documents/GitHub/dcgan/dcgan-local-v{new_vnum:03}"
    os.makedirs(new_dir)

In [ ]:
if cenv == 0:
    try: 
        checkpoint_dir = "/kaggle/working/checkpoints"
        os.makedirs(checkpoint_dir)
        print("Path for checkpoints has been created!")
    except:
        print("Path already exists")
if cenv == 1:
    try:
        checkpoint_dir = f"{new_dir}/checkpoint"
        os.makedirs(checkpoint_dir)
        print("Local path for checkpoints has been created")
    except:
        print("Local path already exists")

In [ ]:
if cenv == 0:
    try:
        gen_img_dir = "/kaggle/working/generated_images"
        os.makedirs(gen_img_dir)
        print("Path for generated images has been created!")
    except:
        print("Path for generating images already exists")
if cenv == 1:
    try:
        gen_img_dir = f"{new_dir}/generated_images"
        os.makedirs(gen_img_dir)
        print("Local path for checkpoints has been created")
    except:
        print("Local path already exists")

**Data**

In [ ]:
# Overwrite if TPU is used
if use_tpu == 'y' or use_tpu == 'Y':
    batch_size = 16 * tpu_strategy.num_replicas_in_sync

In [ ]:
if use_tpu == 'y' or use_tpu == 'Y': 
    # Step 1: Get the credentail from the Cloud SDK
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    user_credential = user_secrets.get_gcloud_credential()
    user_secrets.set_tensorflow_credential(user_credential)

    # Step 3: Use a familiar call to get the GCS path of the dataset
    !gcloud config set project 'solid-topic-344315'
    
    path_root = 'gs://thesis_data_max/classified_data'
    

    
else: 
    if cenv == 0:
        path_root = "/kaggle/input/thesis-data"
        
        #Directory from which checkpoints can be restored if you do GAN in multiple iterations
        checkpoint_restore_directory = '/kaggle/input/dcgan-tweaking-kaggle/checkpoints'
    
    if cenv == 1:
        path_root = "C:/Users/Max/Documents/thesis_data"
        
        #Directory from which checkpoints can be restored if you do GAN in multiple iterations
        checkpoint_restore_directory = ''

Create a dataset from our folder, and rescale the images to the [0-1] range:

In [ ]:
im_si = (ih, iw)

if(ch == 'rgb'):
    chnum = 3
elif(ch == 'grayscale'):
    chnum = 1


In [ ]:
if (use_tpu == 'y' or use_tpu == 'Y'):
    dataset = tf.keras.preprocessing.image_dataset_from_directory(
        GCS_DS_PATH, 
        label_mode = None,
        color_mode = ch,
        image_size = im_si,
        interpolation='bicubic',
        batch_size = batch_size
)
    
if (use_tpu == 'n' or use_tpu == 'N'):
    dataset = tf.keras.preprocessing.image_dataset_from_directory(
        path_root, 
        label_mode = None,
        color_mode = ch,
        image_size = im_si,
        interpolation='bicubic',
        batch_size = batch_size
    )

dataset = dataset.map(lambda x : x / 255.0)

Let's display a sample image:

## Create the discriminator

It maps a 64x64 image to a binary classification score.

In [ ]:
def create_discriminator():
  return keras.Sequential(
      [
        
          layers.Conv2D(ih, kernel_size=ksize, strides=ssize, padding="same",
                        input_shape=(ih, iw, chnum)),
          layers.ReLU(),
          layers.Conv2D(2*ih, kernel_size=ksize, strides=ssize, padding="same"),
          layers.ReLU(),
          layers.Conv2D(2*ih, kernel_size=ksize, strides=ssize, padding="same"),
          layers.ReLU(),
          layers.Flatten(),
          layers.Dropout(0.2),
          layers.Dense(1, activation="sigmoid"),
      ],
      name="discriminator",
  )


## Create the generator

It mirrors the discriminator, replacing `Conv2D` layers with `Conv2DTranspose` layers.

In [ ]:
latent_dim = ih
sih = ih//8
siw = iw//8

def create_generator():
  return keras.Sequential(
      [
          keras.layers.InputLayer(input_shape=(latent_dim)),
          
          layers.Dense(sih * siw * latent_dim),
          layers.Reshape((sih, siw, latent_dim)),
          layers.Conv2DTranspose(latent_dim, kernel_size=ksize, strides=ssize, padding="same"),
          layers.ReLU(),
          layers.Conv2DTranspose(2*latent_dim, kernel_size=ksize, strides=ssize, padding="same"),
          layers.ReLU(),
          layers.Conv2DTranspose(4*latent_dim, kernel_size=ksize, strides=ssize, padding="same"),
          layers.ReLU(),
          layers.Conv2D(chnum, kernel_size=ksize+1, padding="same", activation="sigmoid"),
      ],
      name="generator",
  )


In [ ]:
if use_tpu == 'y' or use_tpu == 'Y':
    with tpu_strategy.scope():
        generator = create_generator()
        discriminator = create_discriminator()
else:  
    generator = create_generator()
    discriminator = create_discriminator()

In [ ]:
discriminator.summary()

In [ ]:
generator.summary()

## Override `train_step`

In [ ]:
class GAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super(GAN, self).__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super(GAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn
        self.d_loss_metric = keras.metrics.Mean(name="d_loss")
        self.g_loss_metric = keras.metrics.Mean(name="g_loss")

    @property
    def metrics(self):
        return [self.d_loss_metric, self.g_loss_metric]

    def train_step(self, real_images):
        # Sample random points in the latent space
        
        batch_size = tf.shape(real_images)[0]
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        # Decode them to fake images
        generated_images = self.generator(random_latent_vectors)

        # Combine them with real images
        combined_images = tf.concat([generated_images, real_images], axis=0)

        # Assemble labels discriminating real from fake images
        labels = tf.concat(
            [tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0
        )
        # Add random noise to the labels - important trick!
        labels += 0.05 * tf.random.uniform(tf.shape(labels))

        # Train the discriminator
        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_images)
            d_loss = self.loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(
            zip(grads, self.discriminator.trainable_weights)
        )

        # Sample random points in the latent space
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        # Assemble labels that say "all real images"
        misleading_labels = tf.zeros((batch_size, 1))

        # Train the generator (note that we should *not* update the weights
        # of the discriminator)!
        with tf.GradientTape() as tape:
            predictions = self.discriminator(self.generator(random_latent_vectors))
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        # Update metrics
        self.d_loss_metric.update_state(d_loss)
        self.g_loss_metric.update_state(g_loss)
        return {
            "d_loss": self.d_loss_metric.result(),
            "g_loss": self.g_loss_metric.result(),
        }
  


**Other GAN Script**

In [ ]:
# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)


In [ ]:
def discriminator_loss(real_output, fake_output, d_loss):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    d_loss.append(total_loss)
    return total_loss

In [ ]:
def generator_loss(fake_output, g_loss):
    fake_loss = cross_entropy(tf.ones_like(fake_output), fake_output)
    g_loss.append(fake_loss)
    return fake_loss

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

## Create a callback that periodically saves generated images

In [ ]:
class GANMonitor(keras.callbacks.Callback):
    def __init__(self, num_img=3, latent_dim=latent_dim):
        self.num_img = num_img
        self.latent_dim = latent_dim

    def on_epoch_end(self, epoch, logs=None):
        start = time.time()
        random_latent_vectors = tf.random.normal(shape=(self.num_img, self.latent_dim))
        generated_images = self.model.generator(random_latent_vectors)
        generated_images *= 255
        generated_images.numpy()
        for i in range(self.num_img):
            img = keras.preprocessing.image.array_to_img(generated_images[i])
            if cenv == 0:
                 img.save(f"{gen_img_dir}/generated_img_%03d_%d.png" % (epoch, i))
            if cenv == 1:
                img.save(f"{gen_img_dir}/generated_img_%03d_%d.png" % (epoch, i))
    
        # Save the model every 15 epochs (WAS 15)
        if (epoch + 1) % 2 == 0:
          checkpoint.save(file_prefix = checkpoint_prefix)



**Restore checkpoints if they exists**

In [ ]:
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=create_generator(),
                                 discriminator=create_discriminator())

In [ ]:
if(os.path.exists(checkpoint_restore_directory)):
    print("A checkpoint was found!")
    ckpt_exists = True # NOT RESTORING FROM CHECKPOINTS
else: 
    print("Checkpoint was not found!")
    ckpt_exists = False

In [ ]:
if ckpt_exists == True and restore == True:
    epochs = 16
    chkpt_restore_path = tf.train.latest_checkpoint(checkpoint_restore_directory)
    print(chkpt_restore_path)
    num_ckpt = chkpt_restore_path[-1:]
    checkpoint.restore(chkpt_restore_path)

In [ ]:
if ckpt_exists == True and restore == True:
    if use_tpu == 'y' or use_tpu == 'Y':
        with tpu_strategy.scope():
            gan = GAN(discriminator=checkpoint.discriminator, generator=checkpoint.generator, latent_dim=latent_dim)

            gan.compile(
            d_optimizer=keras.optimizers.Adam(learning_rate=0.0001), # Was 0.0001
            g_optimizer=keras.optimizers.Adam(learning_rate=0.0001), # Was 0.0001
            loss_fn=keras.losses.BinaryCrossentropy(reduction = tf.keras.losses.Reduction.NONE),
            )

    else: 
        gan = GAN(discriminator=checkpoint.discriminator, generator=checkpoint.generator, latent_dim=latent_dim)

        gan.compile(
        d_optimizer=keras.optimizers.Adam(learning_rate=0.0001), # Was 0.0001
        g_optimizer=keras.optimizers.Adam(learning_rate=0.0001), # Was 0.0001
        loss_fn=keras.losses.BinaryCrossentropy(),
        )

    gan.fit(
      dataset, epochs=epochs, callbacks=[GANMonitor(num_img=10, latent_dim=latent_dim)]
    )

## Train the end-to-end model

In [ ]:
if restore == False:
    epochs = epoch_t  # In practice, use ~100 epochs

    if use_tpu == 'y' or use_tpu == 'Y':
        with tpu_strategy.scope():
            gan = GAN(discriminator=discriminator, generator=generator, latent_dim=latent_dim)

            gan.compile(
            d_optimizer=keras.optimizers.Adam(learning_rate=0.0001), # Was 0.0001
            g_optimizer=keras.optimizers.Adam(learning_rate=0.0001), # Was 0.0001
            loss_fn=keras.losses.BinaryCrossentropy(reduction = tf.keras.losses.Reduction.NONE),
            )

    else: 
        gan = GAN(discriminator=discriminator, generator=generator, latent_dim=latent_dim)

        gan.compile(
        d_optimizer=keras.optimizers.Adam(learning_rate=0.0001), # Was 0.0001
        g_optimizer=keras.optimizers.Adam(learning_rate=0.0001), # Was 0.0001
        loss_fn=keras.losses.BinaryCrossentropy(),
        )

    gan.fit(
      dataset, epochs=epochs, callbacks=[GANMonitor(num_img=10, latent_dim=latent_dim)]
    )

In [ ]:
if restore == True and ckpt_exists == False:
    print("Restore checkpoint cannot be located")

In [ ]:
for x in dataset:
  i = rd.randint(0,26458)
  plt.imshow((x.numpy() * 255).astype("int32")[0], cmap='gray')
  break